# Bonus Bonus
Make sliders that change the parameter values of the distributions in order to visualise their effects

* Bernoulli Distribution
* Beta Distribution
* Categorical Distribution
* Dirichlet Distribution
* Univariate Normal Distribution
* Normal-scaled inverse gamma distribution
* Multivariate Normal distribution
* Normal inverse wishart distribution

In [7]:
from scipy.stats import bernoulli, beta, dirichlet
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Bernoulli Distribution
* Discrete
* Models binary trials
* Describes the situation where there are only two possible outcomes
* $x \in {0,1}$
* Single parameter, $\lambda$ which defines the probability of observing a success $x=1$

Can be expressed as:
$$
\begin{array}{l}{\operatorname{Pr}(x=0)=1-\lambda} \\ {\operatorname{Pr}(x=1)=\lambda}\end{array}
$$

Or alternatively:
$$
\operatorname{Pr}(x)=\lambda^{x}(1-\lambda)^{1-x}
$$

Or:
$$
\operatorname{Pr}(x)=\operatorname{Bern}_{x}[\lambda]
$$

In [14]:
def bern(𝜆):
    data_bern = bernoulli.rvs(size=1000,p=𝜆)
    ax = sns.distplot(data_bern,kde=True,color='crimson',hist_kws={"linewidth": 25,'alpha':1})
    ax.set(xlabel='Bernoulli', ylabel='Frequency')

In [16]:
interact(bern, 𝜆=(0,1,0.01));

interactive(children=(FloatSlider(value=0.0, description='λ', max=1.0, step=0.01), Output()), _dom_classes=('w…

# Beta Distribution
* Continuous distribution defined on single variable 𝜆 where $\lambda \in [0,1]$
* Used for representing uncertainty in the parameter $\lambda$ of the Bernoulli distribution
* Has parameters $(\alpha, \beta)$ whose values determine the expected value so $$
\mathrm{E}[\lambda]=\alpha /(\alpha+\beta)
$$
* $\alpha, \beta \in [0,∞]$

Beta distribution is represented as:
$$
\operatorname{Pr}(\lambda)=\frac{\Gamma[\alpha+\beta]}{\Gamma[\alpha] \Gamma[\beta]} \lambda^{\alpha-1}(1-\lambda)^{\beta-1}
$$

In [23]:
def Beta(𝛼,𝛽):
    data_beta = beta.rvs(𝛼, 𝛽, size=1000)
    ax = sns.distplot(data_beta,kde=True,color='crimson',hist_kws={"linewidth": 25,'alpha':1})
    ax.set(xlabel='Beta Distribution', ylabel='Probability')

In [24]:
interact(Beta, 𝛼=(0,10), 𝛽=(0,10));

interactive(children=(IntSlider(value=5, description='α', max=10), IntSlider(value=5, description='β', max=10)…

In [ ]:
# def categorical(a):
#     data_cat = np.random.multinomial(20, [1/float(a)]*a, size=1)
#     ax = sns.distplot(data_cat,kde=True,color='crimson',hist_kws={"linewidth": 25,'alpha':1})
#     ax.set(xlabel='Categorical', ylabel='Frequency')

# def Dirichlet(a):
#     data_dir = dirichlet.rvs(1,size=1000)
#     ax = sns.distplot(data_dir,kde=True,color='crimson',hist_kws={"linewidth": 25,'alpha':1})
#     ax.set(xlabel='Dirichlet', ylabel='Frequency')

In [ ]:
import matplotlib.tri as tri

_corners = np.array([[0, 0], [1, 0], [0.5, 0.75**0.5]])
_triangle = tri.Triangulation(_corners[:, 0], _corners[:, 1])
_midpoints = [(_corners[(i + 1) % 3] + _corners[(i + 2) % 3]) / 2.0 for i in range(3)]

def xy2bc(xy, tol=1.e-3):
    '''
    Converts 2D Cartesian coordinates to barycentric.
    Arguments:
    `xy`: A length-2 sequence containing the x and y value.
    '''
    s = [(_corners[i] - _midpoints[i]).dot(xy - _midpoints[i]) / 0.75 for i in range(3)]
    return np.clip(s, tol, 1.0 - tol)

def use_draw(a,b,c):
    alpha = np.array([a,b,c])
    draw_pdf_contours(dirichlet(alpha))
    
class Dirichlet(object):
    def __init__(self, alpha):
        '''Creates Dirichlet distribution with parameter `alpha`.'''
        from math import gamma
        from operator import mul
        self._alpha = np.array(alpha)
        self._coef = gamma(np.sum(self._alpha)) / reduce(mul, [gamma(a) for a in self._alpha])
        
    def pdf(self, x):
        '''Returns pdf value for `x`.'''
        from operator import mul
        return self._coef * reduce(mul, [xx ** (aa - 1)
                                         for (xx, aa)in zip(x, self._alpha)])
    def sample(self, N):
        '''Generates a random sample of size `N`.'''
        return np.random.dirichlet(self._alpha, N)
    
def draw_pdf_contours(dist, border=False, nlevels=200, subdiv=8, **kwargs):
    '''Draws pdf contours over an equilateral triangle (2-simplex).
    Arguments:
        `dist`: A distribution instance with a `pdf` method.
        `border` (bool): If True, the simplex border is drawn.
        `nlevels` (int): Number of contours to draw.
        `subdiv` (int): Number of recursive mesh subdivisions to create.
        kwargs: Keyword args passed on to `plt.triplot`.
    '''
    from matplotlib import ticker, cm
    import math

    refiner = tri.UniformTriRefiner(_triangle)
    trimesh = refiner.refine_triangulation(subdiv=subdiv)
    pvals = [dist.pdf(xy2bc(xy)) for xy in zip(trimesh.x, trimesh.y)]

    plt.tricontourf(trimesh, pvals, nlevels, **kwargs)
    plt.axis('equal')
    plt.xlim(0, 1)
    plt.ylim(0, 0.75**0.5)
    plt.axis('off')
    if border is True:
        plt.hold(1)
        plt.triplot(_triangle, linewidth=1)

def plot_points(X, barycentric=True, border=True, **kwargs):
    '''Plots a set of points in the simplex.
    Arguments:
        `X` (ndarray): A 2xN array (if in Cartesian coords) or 3xN array
                       (if in barycentric coords) of points to plot.
        `barycentric` (bool): Indicates if `X` is in barycentric coords.
        `border` (bool): If True, the simplex border is drawn.
        kwargs: Keyword args passed on to `plt.plot`.
    '''
    if barycentric is True:
        X = X.dot(_corners)
    plt.plot(X[:, 0], X[:, 1], 'k.', ms=1, **kwargs)
    plt.axis('equal')
    plt.xlim(0, 1)
    plt.ylim(0, 0.75**0.5)
    plt.axis('off')
    if border is True:
        plt.hold(1)
        plt.triplot(_triangle, linewidth=1)